In [1]:
import pandas as pd
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [2]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [3]:
dataset.drop("User ID", axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [6]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [7]:
dep=dataset[["Purchased"]]
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(indep,dep,test_size=0.30, random_state=0)

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
param_grid={'alpha':[0.1, 0.5, 1.0, 1.5, 2.0],'force_alpha':[True,False],'fit_prior':[True,False]}
grid=GridSearchCV(MultinomialNB(),param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted',error_score='raise')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(error_score='raise', estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [27]:
Y_pred=grid.predict(X_test)
Y_pred

array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0], dtype=int64)

In [28]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
print(cm)

[[68 11]
 [28 13]]


In [29]:
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,Y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.71      0.86      0.78        79
           1       0.54      0.32      0.40        41

    accuracy                           0.68       120
   macro avg       0.62      0.59      0.59       120
weighted avg       0.65      0.68      0.65       120



In [30]:
from sklearn.metrics import f1_score
f1_weighted=f1_score(Y_test,Y_pred,average='weighted')
print(f1_weighted)

0.6482857142857144


In [31]:
from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print("roc_auc score for the best parameter {} : {:.2f}".format(grid.best_params_,roc_score))

roc_auc score for the best parameter {'alpha': 0.1, 'fit_prior': True, 'force_alpha': True} : 0.52


In [32]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014476,0.004730,0.033388,0.003648,0.1,True,True,"{'alpha': 0.1, 'fit_prior': True, 'force_alpha...",0.618850,0.503106,0.630312,0.609571,0.619155,0.596199,0.047008,1
1,0.007586,0.003081,0.021682,0.003605,0.1,True,False,"{'alpha': 0.1, 'fit_prior': True, 'force_alpha...",0.618850,0.503106,0.630312,0.609571,0.619155,0.596199,0.047008,1
2,0.010323,0.000385,0.016616,0.005276,0.1,False,True,"{'alpha': 0.1, 'fit_prior': False, 'force_alph...",0.527697,0.472527,0.555872,0.578065,0.558458,0.538524,0.036700,15
3,0.004570,0.005097,0.018708,0.004353,0.1,False,False,"{'alpha': 0.1, 'fit_prior': False, 'force_alph...",0.527697,0.472527,0.555872,0.578065,0.558458,0.538524,0.036700,15
4,0.008195,0.003925,0.020313,0.000140,0.5,True,True,"{'alpha': 0.5, 'fit_prior': True, 'force_alpha...",0.618850,0.503106,0.630312,0.609571,0.619155,0.596199,0.047008,1
5,0.008116,0.004058,0.020264,0.000063,0.5,True,False,"{'alpha': 0.5, 'fit_prior': True, 'force_alpha...",0.618850,0.503106,0.630312,0.609571,0.619155,0.596199,0.047008,1
6,0.006085,0.004922,0.020198,0.006405,0.5,False,True,"{'alpha': 0.5, 'fit_prior': False, 'force_alph...",0.527697,0.472527,0.555872,0.578065,0.558458,0.538524,0.036700,15
7,0.008072,0.004036,0.020920,0.000905,0.5,False,False,"{'alpha': 0.5, 'fit_prior': False, 'force_alph...",0.527697,0.472527,0.555872,0.578065,0.558458,0.538524,0.036700,15
8,0.010489,0.000718,0.016147,0.004978,1.0,True,True,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,5
9,0.012140,0.004024,0.016481,0.005251,1.0,True,False,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,5
